In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:54 of 0:12:54 remaining]
Chain 1:  10% [0:00:55 of 0:01:01 remaining]
Chain 1:  20% [0:00:59 of 0:01:14 remaining]
Chain 1:  30% [0:00:55 of 0:01:19 remaining]
Chain 1:  40% [0:00:47 of 0:01:19 remaining]
Chain 1:  50% [0:00:39 of 0:01:18 remaining]
Chain 1:  60% [0:00:31 of 0:01:18 remaining]
Chain 1:  70% [0:00:23 of 0:01:18 remaining]
Chain 1:  80% [0:00:16 of 0:01:18 remaining]
Chain 1:  90% [0:00:08 of 0:01:18 remaining]
Chain 1: 100% [0:00:00 of 0:01:20 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:49 of 0:00:54 remaining]
Chain 2:  20% [0:00:52 of 0:01:05 remaining]
Chain 2:  30% [0:00:47 of 0:01:07 remaining]
Chain 2:  40% [0:00:41 of 0:01:08 remaining]
Chain 2:  50% [0:00:34 of 0:01:09 remaining]
Chain 2:  60% [0:00:28 of 0:01:10 remaining]
Chain 2:  70% [0:00:21 of 0:01:09 remaining]
Chain 2:  80% [0:00:14 of 0:01:09 remaining]
Chain 2:  90% [0:00:07 of 0:01:10 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.4493 112.801 0.928952; 24.5019 112.801 0.928952; … ; 29.0218 116.054 0.857501; 29.5912 116.059 0.855871]

[27.2122 113.183 0.914063; 26.8716 113.719 0.914682; … ; 23.0684 113.304 0.913253; 24.2408 113.244 0.918675]

[22.4222 114.852 0.881698; 31.5334 113.989 0.902371; … ; 24.9668 114.18 0.889714; 26.3888 114.18 0.889714]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD       Naive SE        MCSE        ESS   
     s2  26.16629544 1.99460008 0.0121387495 0.02079171933 9000.0000
beta[1] 113.86752723 1.88927944 0.0114977885 0.03438235594 3019.4034
beta[2]   0.90521411 0.04154291 0.0002528221 0.00075307744 3043.0883

Quantiles:
            2.5%       25.0%       50.0%       75.0%        97.5%   
     s2  22.5524892  24.7718918  26.0682694  27.4511863  30.31838370
beta[1] 110.1535102 112.5729497 113.8894639 115.1568906 117.50771585
beta[2]   0.8249097   0.8770099   0.9046399   0.9331817   0.98694885



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*